In [8]:

# temel sikayetvar veri kazıma

import aiohttp
import asyncio
from bs4 import BeautifulSoup
import pandas as pd

headers = {'User-Agent': 'Mozilla/5.0'}
sikayetler = pd.DataFrame(columns=['Marka', 'Sayfa Sayisi', 'Link', 'Sikayet'])
async def fetch(session, url):
    try:
        async with session.get(url, headers=headers) as response:
            if response.status == 200:
                return await response.text()
            else:
                return None
    except Exception as e:
        print(f"Error fetching {url}: {e}")
        return None

async def marka_sayfasi(session, marka, sayfa):
    url = f'https://www.sikayetvar.com/{marka}?page={sayfa}'
    html = await fetch(session, url)
    if html:
        return BeautifulSoup(html, 'html.parser')
    return None

async def sikayet_sayfasi(session, link):
    url = f'https://www.sikayetvar.com{link}'
    html = await fetch(session, url)
    if html:
        return BeautifulSoup(html, 'html.parser')
    return None

async def scrape_sikayetler(marka, ilk_sayfa, son_sayfa):
    async with aiohttp.ClientSession() as session:

        for i in range(ilk_sayfa, son_sayfa + 1):
            print(f"{i}. sayfa taranıyor...")
            soup = await marka_sayfasi(session, marka, i)
            if not soup:
                print(f"{i}. sayfada veri çekilemedi.")
                await asyncio.sleep(2)  
                continue

            sikayet_linkler = [l['href'] for l in soup.find_all('a', class_='complaint-layer') if l['href'].count(marka) > 0]
            sayfada_kazinan = len(sikayet_linkler)
            
            tasks = [sikayet_sayfasi(session, j) for j in sikayet_linkler]
            results = await asyncio.gather(*tasks)

            for soup_sikayet, j in zip(results, sikayet_linkler):
                if soup_sikayet and soup_sikayet.find('div', class_="complaint-detail-description"):
                    text = " ".join(soup_sikayet.find('div', class_="complaint-detail-description").text.split(" ")[1:])
                    sikayetler.loc[len(sikayetler)] = [marka, i, j, text.strip()]
                await asyncio.sleep(1)  # Her şikayet sayfası çekiminden sonra bekleme süresi

            print(f"{i}. sayfa tarandı. {sayfada_kazinan} veri çekildi.")
            await asyncio.sleep(2)  # Sayfalar arasında bekleme süresi
        
        print("bitti")

marka = 'apple'
ilk_sayfa = 20
son_sayfa = 50

await scrape_sikayetler(marka, ilk_sayfa, son_sayfa)


20. sayfa taranıyor...
20. sayfa tarandı. 22 veri çekildi.
21. sayfa taranıyor...
21. sayfa tarandı. 23 veri çekildi.
22. sayfa taranıyor...
22. sayfada veri çekilemedi.
23. sayfa taranıyor...
23. sayfa tarandı. 24 veri çekildi.
24. sayfa taranıyor...
24. sayfada veri çekilemedi.
25. sayfa taranıyor...
25. sayfa tarandı. 22 veri çekildi.
26. sayfa taranıyor...
26. sayfa tarandı. 22 veri çekildi.
27. sayfa taranıyor...
27. sayfa tarandı. 22 veri çekildi.
28. sayfa taranıyor...
28. sayfa tarandı. 24 veri çekildi.
29. sayfa taranıyor...
29. sayfa tarandı. 21 veri çekildi.
30. sayfa taranıyor...
30. sayfa tarandı. 23 veri çekildi.
31. sayfa taranıyor...
31. sayfa tarandı. 23 veri çekildi.
32. sayfa taranıyor...
32. sayfa tarandı. 24 veri çekildi.
33. sayfa taranıyor...
33. sayfa tarandı. 23 veri çekildi.
34. sayfa taranıyor...
34. sayfa tarandı. 23 veri çekildi.
35. sayfa taranıyor...
35. sayfa tarandı. 23 veri çekildi.
36. sayfa taranıyor...
36. sayfa tarandı. 23 veri çekildi.
37. sayfa t

CancelledError: 

In [6]:
sikayetler.to_csv('apple-sikayetvar1.csv')

In [3]:
import pandas as pd   #csv leri birleştirme

dosyalar = ['sikayetler-13.csv', 'sikayetler-12.csv', 'sikayetler-11.csv', 'sikayetler-14.csv']

birlesik_df = pd.DataFrame()

for dosya in dosyalar:
    df = pd.read_csv(dosya)
    birlesik_df = pd.concat([birlesik_df, df], ignore_index=True)

birlesik_df.to_csv('tum_sikayetler.csv', index=False)

print("Tüm dosyalar başarıyla birleştirildi ve 'tum_sikayetler.csv' dosyasına kaydedildi.")


Tüm dosyalar başarıyla birleştirildi ve 'tum_sikayetler.csv' dosyasına kaydedildi.


In [2]:
import pandas as pd

df = pd.read_csv("apple-sikayetvar.csv")

df = df.drop_duplicates(subset=['Sikayet'])

df.to_csv("sikayetvar-paycell_duzenlenmis.csv", index=False)

print("Dosya başarıyla düzenlendi.")


Dosya başarıyla düzenlendi.


In [4]:
import pandas as pd

paycell_df = pd.read_csv("steam-sikayetvar.csv")
tum_sikayetler_df = pd.read_csv("paycell1.csv")

paycell_df.rename(columns={'Şikayet': 'Metin'}, inplace=True)


paycell_df.to_csv("steam-sikayetvar-guncellenmis.csv", index=False)

print("Dosyalar başarıyla birleştirildi.")

Dosyalar başarıyla birleştirildi.


In [9]:
import pandas as pd
import json

csv_file_path = r'C:\Users\Administrator\Downloads\apple-sikayetvar1.csv'

df = pd.read_csv(csv_file_path)

sikayetler = df['Sikayet'].tolist()

data = [{"sentence": sikayet} for sikayet in sikayetler]

with open('unlabeled_apple.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)



In [14]:
import json

def transform_data(data):
    transformed_data = []
    
    for item in data:
        sentence = item['sentence']
        results = item['results']
        
        entity_list = [result['entity'] for result in results]
        
        results_list = [{'entity': result['entity'], 'sentiment': result['sentiment']} for result in results]
        
        transformed_item = {
            'sentence': sentence,
            'entity_list': entity_list,
            'results': results_list
        }
        
        transformed_data.append(transformed_item)
    
    return transformed_data

input_file = 'turkcell-superonline-fiberinternet.json'
output_file = 'turkcell-superonline-fiberinternet-labeled.json'

with open(input_file, 'r', encoding='utf-8') as file:
    data = json.load(file)

transformed_data = transform_data(data)

with open(output_file, 'w', encoding='utf-8') as file:
    json.dump(transformed_data, file, ensure_ascii=False, indent=4)

print(f"Dönüştürülmüş veriler '{output_file}' dosyasına kaydedildi.")


Dönüştürülmüş veriler 'turkcell-superonline-fiberinternet-labeled.json' dosyasına kaydedildi.


In [3]:
#Yanlış veya eksik etiketlenmiş verilerin kontrolü için

import json

with open('dosya.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

for item in data:
    if 'sentence' not in item:
        item['sentence'] = "default sentence"

    if 'results' not in item:
        item['results'] = [{"entity": "default entity", "sentiment": "default sentiment"}]
    else:
        if 'entity_list' not in item:
            item['entity_list'] = [result['entity'] for result in item['results'] if 'entity' in result]

        sentence = item['sentence'].lower()
        sentence_words = sentence.split()

        halusinations = []
        results_entities = [result['entity'].lower() for result in item['results'] if 'entity' in result]
        entity_list_entities = [entity.lower() for entity in item['entity_list']]

        for result in item['results']:
            if 'entity' not in result:
                result['entity'] = "default entity"
            if 'sentiment' not in result:
                result['sentiment'] = "default sentiment"       # olmayan result attributelerine default değerini atıyor.
            entity = result['entity'].lower()
            if entity not in sentence:
                result['halusinasyon'] = True
                halusinations.append(result['entity'])
        
        for entity in item['entity_list']:
            entity_lower = entity.lower()
            if entity_lower not in sentence:
                if 'halusinasyon' not in item:
                    item['halusinasyon'] = []
                item['halusinasyon'].append(entity)
                if entity not in halusinations:
                    halusinations.append(entity)

        for entity in entity_list_entities:
            if entity not in results_entities:      
                if 'halusinasyon' not in item:
                    item['halusinasyon'] = []
                item['halusinasyon'].append(entity)
                if entity not in halusinations:
                    halusinations.append(entity)

        if halusinations:
            item['halusinasyon'] = halusinations

with open('yeniDosya.json', 'w', encoding='utf-8') as file: # yeni kaydedilecek dosya
    json.dump(data, file, ensure_ascii=False, indent=4)
